In [ ]:
import pandas as pd
from sqlalchemy import create_engine 
import math
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import column, grid
from bokeh.models import Band, ColumnDataSource
from sqlalchemy import create_engine, MetaData, Table, Column, Float, String


In [ ]:
class BasisOrganizer:
    try:
        def __init__(self, Location_Of_the_CSV):
            
            self._organizers = []

            
            try:
                self._organizer_data = pd.read_csv(Location_Of_the_CSV)
            except FileNotFoundError:
                print("Error reading the CSV file {}".format(Location_Of_the_CSV))
                raise

            
            x_data = self._organizer_data["x"]

            
            for lable_of_column, value_of_column in self._organizer_data.items():
                if "x" in lable_of_column:
                    continue
                
                subgroup = pd.concat([x_data, value_of_column], axis=1)
                organizer = Organizer.from_dataframe(lable_of_column, subgroup)
                self._organizers.append(organizer)


        def to_sql(self, name_of_CSV, suffix):
            
            engine = create_engine('sqlite:///{}.db'.format(name_of_CSV), echo=False)

            
            Dupliacte_of_organizer_data = self._organizer_data.copy()
            Dupliacte_of_organizer_data.columns = [title.capitalize() + suffix for title in Dupliacte_of_organizer_data.columns]
            Dupliacte_of_organizer_data.set_index(Dupliacte_of_organizer_data.columns[0], inplace=True)

            Dupliacte_of_organizer_data.to_sql(
                name_of_CSV,
                engine,
                if_exists="replace",
                index=True,
            )

        @property
        def organizers(self):
            
            return self._organizers

        def __iter__(self):
            
            return BasisOrganizerRepeat(self)

        def __repr__(self):
            return "Contains {} number of organizers".format(len(self.organizers))
    except Exception as e:
        print("Error in BasisOrganizer"+ str(e))

class BasisOrganizerRepeat():
    try:
        def __init__(self, organizer_administrator):
            
            self._index = 0
            self._organizer_administrator = organizer_administrator

        def __next__(self):
            
            if self._index < len(self._organizer_administrator.organizers):
                value_requested = self._organizer_administrator.organizers[self._index]
                self._index = self._index + 1
                return value_requested
            raise StopIteration
    except Exception as e:
        print("Error in BasisOrganizerRepeat"+ str(e))


class Organizer:
    try:
        def __init__(self, title):
            
            self._title = title
            self.dataframe = pd.DataFrame()

        def Finding_y_driven_from_x(self, x):
            
            search_key = self.dataframe["x"] == x
            try:
                return self.dataframe.loc[search_key].iat[0, 1]
            except IndexError:
                raise IndexError


        @property
        def title(self):
            
            return self._title

        def __iter__(self):
            return OrganizerRepeat(self)

        def __sub__(self, other):
            
            diff = self.dataframe - other.dataframe
            return diff

        @classmethod
        def from_dataframe(cls, title, dataframe):
            
            organizer = cls(title)
            organizer.dataframe = dataframe
            organizer.dataframe.columns = ["x", "y"]
            return organizer

        def __repr__(self):
            return "Organizer for {}".format(self.title)
    except Exception as e:
        print("Error in Organizer"+ str(e))

class IdealOrganizer(Organizer):
    try:
        def __init__(self, organizer, training_organizer, error):
            
            super().__init__(organizer.title)
            self.dataframe = organizer.dataframe

            self.training_organizer = training_organizer
            self.error = error
            self._forbearance_value = 1
            self._forbearance = 1

        def _decide_largest_variance(self, ideal_Organizer, train_Organizer):
            
            spans = train_Organizer - ideal_Organizer
            spans["y"] = spans["y"].abs()
            large_variance = max(spans["y"])
            return large_variance

        @property
        def forbearance(self):
            
            self._forbearance = self.forbearance_factor * self.large_variance
            return self._forbearance

        @forbearance.setter
        def forbearance(self, value):

            self._forbearance = value

        @property
        def forbearance_factor(self):
            
            return self._forbearance_value

        @forbearance_factor.setter
        def forbearance_factor(self, value):
            self._forbearance_value = value

        @property
        def large_variance(self):
            
            large_variance = self._decide_largest_variance(self, self.training_organizer)
            return large_variance
    except Exception as e:
        print("Error in IdealOrganizer"+ str(e))


class OrganizerRepeat:
    try:
        def __init__(self, organizer):
            
            self._organizer = organizer
            self._index = 0

        def __next__(self):
            
            if self._index < len(self._organizer.dataframe):
                value_requested_series = (self._organizer.dataframe.iloc[self._index])
                point = {"x": value_requested_series.x, "y": value_requested_series.y}
                self._index += 1
                return point
            raise StopIteration
    except Exception as e:
        print("Error in OrganizerRepeat"+ str(e))


In [ ]:

def P_Ideal_organizers(ideal_Organizers, name_of_CSV):
    
    ideal_Organizers.sort(key = lambda ideal_Organizer: ideal_Organizer.training_organizer.title, reverse=False)
    plots = []
    for ideal_Organizer in ideal_Organizers:
        p = plot_Chart_from_two_Organizer(line_Organizer=ideal_Organizer, scatter_Organizer=ideal_Organizer.training_organizer,
                                        quadratic_error=ideal_Organizer.error)
        plots.append(p)
    output_file("{}.html".format(name_of_CSV))
    
    show(column(*plots))


def plot_Bullets_with_their_ideal_Organizer(bullets_with_class, name_of_CSV):
    
    plots = []
    for index, item in enumerate(bullets_with_class):
        if item["classification"] is not None:
            p = plot_grading(item["point"], item["classification"])
            plots.append(p)
    output_file("{}.html".format(name_of_CSV))
    show(column(*plots))


def plot_Chart_from_two_Organizer(scatter_Organizer, line_Organizer, quadratic_error):
    
    f1_dataframe = scatter_Organizer.dataframe
    f1_name = scatter_Organizer.title

    f2_dataframe = line_Organizer.dataframe
    f2_name = line_Organizer.title

    quadratic_error = round(quadratic_error, 2)
    p = figure(title="train model {} vs ideal {}. Total squared error = {}".format(f1_name, f2_name, quadratic_error),
            x_axis_label='x', y_axis_label='y')
    p.scatter(f1_dataframe["x"], f1_dataframe["y"], fill_color="red", legend_label="Train")
    p.line(f2_dataframe["x"], f2_dataframe["y"], legend_label="Ideal", line_width=2)
    return p


def plot_grading(point, ideal_Organizer):
    
    if ideal_Organizer is not None:
        try:
            classification_organizer_dataframe = ideal_Organizer.dataframe

            point_str = "({},{})".format(point["x"], round(point["y"], 2))
            title = "point {} with classification: {}".format(point_str, ideal_Organizer.title)

            p = figure(title=title, x_axis_label='x', y_axis_label='y')

            
            p.line(classification_organizer_dataframe["x"], classification_organizer_dataframe["y"],
                    legend_label="classification Organizer", line_width=2, line_color='black')

            
            criterion = ideal_Organizer.forbearance
            classification_organizer_dataframe['upper'] = classification_organizer_dataframe['y'] + criterion
            classification_organizer_dataframe['lower'] = classification_organizer_dataframe['y'] - criterion

            source = ColumnDataSource(classification_organizer_dataframe.reset_index())

            band = Band(base='x', lower='lower', upper='upper', source=source, level='underlay',
                fill_alpha=0.3, line_width=1, line_color='green', fill_color="green")

            p.add_layout(band)

            
            p.scatter([point["x"]], [round(point["y"], 4)], fill_color="red", legend_label="Test point", size=8)

            return p
        except Exception as e:
            print("Error in plot_grading"+ str(e))


In [ ]:

def reduce_loss(training_organizer, inventory_contender_organizers, losing_organizer):
    try:    
        function_with_smallest_error = None
        smallest_error = None
        for organizer in inventory_contender_organizers:
            error = losing_organizer(training_organizer, organizer)
            if ((smallest_error == None) or error < smallest_error):
                smallest_error = error
                function_with_smallest_error = organizer

        ideal_Organizer = IdealOrganizer(organizer=function_with_smallest_error, training_organizer=training_organizer,
                            error=smallest_error)
        return ideal_Organizer
    except Exception as e:
            print("Error in reduce_loss"+ str(e))


def classification_search(point, ideal_Organizers):
    
    current_lowest_classification = None
    current_lowest_span = None

    for ideal_Organizer in ideal_Organizers:
        try:
            locate_y_in_classification = ideal_Organizer.Finding_y_driven_from_x(point["x"])
        except IndexError:
            print("This point is not in the classification Organizer")
            raise IndexError

        
        span = abs(locate_y_in_classification - point["y"])

        if (abs(span < ideal_Organizer.forbearance)):
            
            if ((current_lowest_classification == None) or (span < current_lowest_span)):
                current_lowest_classification = ideal_Organizer
                current_lowest_span = span

    return current_lowest_classification, current_lowest_span


In [ ]:
def write_variance_outcome_to_sqlite(result):
    try:
        engine = create_engine('sqlite:///{}.db'.format("Final_mapping"), echo=False)
        meadata = MetaData()
        
        Final_mapping =Table("Final_mapping", meadata,
                    Column('X (Test organizer)', Float, primary_key=False),
                    Column('Y (Test organizer)', Float),
                    Column('Delta Y (Test organizer)', Float),
                    Column('No. of ideal organizer', String(50))
        )
        meadata.create_all(engine)

        execute_map=[]
        for item in result:
            point = item["point"]
            classification  = item["classification"]
            delta_y = item["delta_y"]
            classification_name  = None
            if classification  is not None:
                classification_name = classification.title.replace("Y","N")
            else:
                classification_name = "-"
                delta_y =-1
            res = {
                'X (Test organizer)': point["x"],
                'Y (Test organizer)': point["y"],
                'Delta Y (Test organizer)': delta_y,
                'No. of ideal organizer': classification_name
            }
            execute_map.append(res)

        with engine.begin() as connection:
            connection.execute(Final_mapping.insert(), execute_map)
    except Exception as e:
            print("Error in write_variance_outcome_to_sqlite"+ str(e))

In [ ]:
def quadratic_error(first_function, second_function):
    try:    
        spans = second_function - first_function
        spans["y"] = spans["y"] ** 2
        entire_variance = sum(spans["y"])
        return entire_variance
    except Exception as e:
            print("Error in quadratic_error"+ str(e))


In [ ]:


GAIN_PART = math.sqrt(2)

if __name__ == '__main__':
    
    ideal_directory = "data/ideal.csv"
    train_directory = "data/train.csv"
    test_directory = "data/test.csv"

    
    contender_ideal_Organizer_administrator = BasisOrganizer(Location_Of_the_CSV=ideal_directory)
    train_Organizer_administrator = BasisOrganizer(Location_Of_the_CSV=train_directory)

    
    train_Organizer_administrator.to_sql(name_of_CSV="training", suffix=" (training func)")
    contender_ideal_Organizer_administrator.to_sql(name_of_CSV="ideal", suffix=" (ideal organizer)")

    
    ideal_Organizers = []
    for train_Organizer in train_Organizer_administrator:
        
        ideal_Organizer = reduce_loss(training_organizer=train_Organizer,
                                    inventory_contender_organizers=contender_ideal_Organizer_administrator.organizers,
                                    losing_organizer=quadratic_error)
        ideal_Organizer.forbearance_factor = GAIN_PART
        ideal_Organizers.append(ideal_Organizer)

    
    P_Ideal_organizers(ideal_Organizers, "train_and_ideal")

    
    
    test_organizer_administrator = BasisOrganizer(Location_Of_the_CSV=test_directory)
    test_organizer = test_organizer_administrator.organizers[0]

    PW_ideal_Organizer = []
    for point in test_organizer:
        ideal_Organizer, delta_y = classification_search(point=point, ideal_Organizers=ideal_Organizers)
        result = {"point": point, "classification": ideal_Organizer, "delta_y": delta_y}
        PW_ideal_Organizer.append(result)

    
    plot_Bullets_with_their_ideal_Organizer(PW_ideal_Organizer, "point_and_ideal")

    
    write_variance_outcome_to_sqlite(PW_ideal_Organizer)
    
    